In [3]:
import pickle as pkl
import tqdm.notebook as tqdm
import cv2
import torch
import time
import math

from numpy import random
import os
import tqdm.notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance 
from IPython import display
from ipywidgets import Output

import custom_utils
torch.backends.cudnn.benchmark = True

aircraft-detection custom utils


In [4]:
%matplotlib inline

In [5]:
#static camera properties
touchdown_target_lat_lon = [1.3541354199301814, 103.97961848373048]

data_indexes = [1,2]
data_sources = []
t = 1
r = 1
w = 1

for data_index in data_indexes:
    data_sources.append([f'dataset\\t{t}\\r{r}\\w{w}\\{data_index}\\data.pkl', f'dataset\\t{t}\\r{r}\\w{w}\\{data_index}\\video.avi'])
    
acd = custom_utils.aircraft_camera_data(data_sources = data_sources, touchdown_target_lat_lon = touchdown_target_lat_lon, model_sources = ["trained_mlp\\wsss_mono_view_1.pt", "trained_mlp\\wsss_mono_view_2.pt", "trained_mlp\\wsss_stereo.pt"])

acd.initiate_yolov7()
acd.initiate_mlp_dual()

Adding autoShape... 


In [17]:
def auto_segmentation(image, target_segment = [3,2]):
    images = []
    image_size = image.shape
    target_size = [math.floor(image_size[1]/target_segment[0]), math.floor(image_size[0]/target_segment[1])]
    for j in range(0, target_segment[1]):
        crop_height = target_size[1]
        if j == (target_segment[1]-1):
            crop_height += image_size[0]%target_segment[1]  
        for i in range(0,target_segment[0]):
            crop_width = target_size[0]
            if i == (target_segment[0]-1):
                crop_width += image_size[1]%target_segment[0]
            images.append(np.ascontiguousarray(image[j*target_size[1]:j*target_size[1]+crop_height, i*target_size[0]:i*target_size[0]+crop_width]))
    return images

In [18]:
%timeit auto_segmentation(acd.get_frame_from_video_synced(synced_index = 100)[0])

209 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
